In [1]:
from typing import List, Dict
import pandas as pd
import pyterrier as pt
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict
from datasets import Dataset
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import DataCollatorWithPadding
from datetime import datetime
from transformers import pipeline
import os
import csv

pt.init()

ModuleNotFoundError: No module named 'transformers'

In [2]:
def load_dataset(
    files: Dict[str, str]={"train": "Data/webis-clickbait-22/train.jsonl", "validation":"Data/webis-clickbait-22/validation.jsonl"}) -> pd.DataFrame:
    mapping: Dict[str, str]={'passage': 0, 'phrase':1, 'multi':2}

    dataset = {}
    
    def load_data(file: str):
        df = pd.read_json(file, lines=True)
        return df


    for split in list(files.keys()):
        dataset[split] = load_data(files[split])

        
    return dataset

In [3]:
dataset = load_dataset()

In [4]:
for _, row in dataset["train"].iterrows():
    break

# Type classification

In [14]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Multi model prediction
checkpoint = "path to model"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
mapping = {0: 'passage', 1: 'phrase', 2: 'multi'}

def predict_one(text, model):
    tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**tokenized).logits
    predicted_class_id = logits.argmax().item()
    return mapping[predicted_class_id]

predicted_tag = predict_one(text, model)

ModuleNotFoundError: No module named 'transformers'

In [ ]:
# one vs rest prediction
checkpoint = {"passage": "path to model", "phrase": "path to model", "multi": "path to model"}
tokenizer = AutoTokenizer.from_pretrained(checkpoint["passage"])
models = {
    "passage": AutoModelForSequenceClassification.from_pretrained(checkpoint["passage"], num_labels=2),
    "phrase": AutoModelForSequenceClassification.from_pretrained(checkpoint["phrase"], num_labels=2),
    "multi": AutoModelForSequenceClassification.from_pretrained(checkpoint["multi"], num_labels=2)
}

def predict_one_multi(text, model):
    def predict_probability(text, model):
        tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            logits = model(**tokenized).logits
        return logits.argmax()


    probabilities = {}
    for tag_name, model in models.items():
        probability = predict_probability(text, model)
        probabilities[tag_name] = probability

    return max(probabilities, key=probabilities.get)


predicted_tag = predict_one_multi(text, models)

# Passage retrieval

In [11]:
passages = pd.DataFrame({
    "qid": "q1",
    "query": [row["postText"][0] for _ in range(len(row["targetParagraphs"]))],
    "docno": ["d"+str(i+1) for i in range(len(row["targetParagraphs"]))],
    "text": row["targetParagraphs"], 
    })
textscorer = pt.text.scorer(body_attr="text", wmodel="BM25")
rtr = textscorer.transform(passages)

18:30:08.449 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


In [12]:
rtr.sort_values("rank", ascending=True)

,qid,docno,text,rank,score,query
0,q1,d1,It’ll be just like old times this weekend for ...,0,2.692426,"Wes Welker Wanted Dinner With Tom Brady, But P..."
2,q1,d3,"But Brady’s definition of ""catching up"" involv...",1,0.385074,"Wes Welker Wanted Dinner With Tom Brady, But P..."
3,q1,d4,"""I hit him up to do dinner Saturday night. He’...",2,-0.355864,"Wes Welker Wanted Dinner With Tom Brady, But P..."
4,q1,d5,Welker was one of Brady’s favorite targets for...,3,-0.795285,"Wes Welker Wanted Dinner With Tom Brady, But P..."
5,q1,d6,"""That’s the only thing we really have planned,...",4,-0.875628,"Wes Welker Wanted Dinner With Tom Brady, But P..."
1,q1,d2,Welker revealed Friday morning on a Miami radi...,5,-1.656121,"Wes Welker Wanted Dinner With Tom Brady, But P..."
6,q1,d7,Maybe Brady will put a good word in for Welker...,6,-2.125572,"Wes Welker Wanted Dinner With Tom Brady, But P..."


# Generation decision

In [ ]:
if tags[0] == "phrase":
    spoiler = answer_sequence(rtr.sort_values("rank", ascending=True).iloc[0]["text"], rtr.sort_values("rank", ascending=True).iloc[0]["query"])

elif tags[0] == "passage":
    spoiler = answer_sequence(rtr.sort_values("rank", ascending=True).iloc[0]["text"], rtr.sort_values("rank", ascending=True).iloc[0]["query"])
    
elif tags[0] == "multi":
    num_answers = estimate_answers(row["postText"][0])

In [ ]:
qa_model = pipeline("question-answering", model="mirbostani/bert-base-uncased-finetuned-newsqa")

def answer_sequence(passages: str, post: str) -> str:
    results = []
    res = qa_model(question = post, context = passages)
    return res

In [ ]:
def estimate_answers() -> int:
    return 1

In [36]:
dataset["train"]["tags_clean"]=dataset["train"]["tags"].explode()

In [49]:
paragraphs = dataset["train"][dataset["train"]["tags_clean"]=="passage"][["targetParagraphs", "spoiler"]].explode("targetParagraphs").explode("spoiler")

In [51]:
paragraphs[paragraphs["targetParagraphs"]==paragraphs["spoiler"]]

,targetParagraphs,spoiler
26,The off-duty officer ended up eating at the ch...,The off-duty officer ended up eating at the ch...
107,A man looking to blame President-Elect Donald ...,A man looking to blame President-Elect Donald ...
129,"Ariel Winter, who plays Alex on ""Modern Family...","Ariel Winter, who plays Alex on ""Modern Family..."
135,"""Back then, I still felt I was trying to figur...","""Back then, I still felt I was trying to figur..."
147,"She took the kitten home and named it Bruce, a...","She took the kitten home and named it Bruce, a..."
...,...,...
3121,"You'll still be able to get your PERinaise, le...","You'll still be able to get your PERinaise, le..."
3134,"According to The Sun, a total lunar eclipse wi...","According to The Sun, a total lunar eclipse wi..."
3149,An online feud between pop stars Rihanna and A...,An online feud between pop stars Rihanna and A...
3174,And while there are good signs for Trump in th...,And while there are good signs for Trump in th...


In [54]:
dataset["train"]["tags_clean"].groupby(dataset["train"]["tags_clean"]).count()

tags_clean
multi       559
passage    1274
phrase     1367
Name: tags_clean, dtype: int64

In [55]:
paragraphs

,targetParagraphs,spoiler
0,It’ll be just like old times this weekend for ...,how about that morning we go throw?
0,Welker revealed Friday morning on a Miami radi...,how about that morning we go throw?
0,"But Brady’s definition of ""catching up"" involv...",how about that morning we go throw?
0,"""I hit him up to do dinner Saturday night. He’...",how about that morning we go throw?
0,Welker was one of Brady’s favorite targets for...,how about that morning we go throw?
...,...,...
3195,One interesting note from last night’s gatheri...,it hasn’t necessarily taken the wind out of Yo...
3195,"""Facebook’s focus on real identity allows for ...",it hasn’t necessarily taken the wind out of Yo...
3196,"The Kansas City, Kansas Police Department are ...","It read, ""Thanks for keeping us safe."""
3196,KCK Police Department’s Facebook posted furthe...,"It read, ""Thanks for keeping us safe."""
